In [28]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("psparks/instacart-market-basket-analysis")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Qamar Hasan\.cache\kagglehub\datasets\psparks\instacart-market-basket-analysis\versions\1


In [29]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

In [43]:
aisles = pd.read_csv(f"{path}/aisles.csv")
dept = pd.read_csv(f"{path}/departments.csv")
products = pd.read_csv(f"{path}/products.csv")
orders = pd.read_csv(f"{path}/orders.csv")
order_prior = pd.read_csv(f"{path}/order_products__prior.csv")
order_train = pd.read_csv(f"{path}/order_products__train.csv")

# Preview first 5 rows of one file
aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [31]:
# Merge prior orders with products
order_products = pd.merge(order_prior, products, on="product_id", how="left")

# Merge with orders to get user/order info
order_products = pd.merge(order_products, orders[['order_id', 'user_id']], on="order_id", how="left")

product_counts = order_products['product_name'].value_counts()

order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id
0,2,33120,1,1,Organic Egg Whites,86,16,202279
1,2,28985,2,1,Michigan Organic Kale,83,4,202279
2,2,9327,3,0,Garlic Powder,104,13,202279
3,2,45918,4,1,Coconut Butter,19,13,202279
4,2,30035,5,0,Natural Sweetener,17,13,202279


In [44]:
print("Total number of rows in order_products:", order_products.shape[0])

Total number of rows in order_products: 32434489


In [45]:
product_counts.unique().sum()

np.int64(23430661)

In [46]:
# Group by order_id to create transaction lists
transactions = order_products.groupby("order_id")["product_name"].apply(list).values.tolist()

print(transactions[0:5]) 


[['Organic Egg Whites', 'Michigan Organic Kale', 'Garlic Powder', 'Coconut Butter', 'Natural Sweetener', 'Carrots', 'Original Unflavored Gelatine Mix', 'All Natural No Stir Creamy Almond Butter', 'Classic Blend Cole Slaw'], ['Total 2% with Strawberry Lowfat Greek Strained Yogurt', 'Unsweetened Almondmilk', 'Lemons', 'Organic Baby Spinach', 'Unsweetened Chocolate Almond Breeze Almond Milk', 'Organic Ginger Root', 'Air Chilled Organic Boneless Skinless Chicken Breasts', 'Organic Ezekiel 49 Bread Cinnamon Raisin'], ['Plain Pre-Sliced Bagels', 'Honey/Lemon Cough Drops', 'Chewy 25% Low Sugar Chocolate Chip Granola', 'Oats & Chocolate Chewy Bars', "Kellogg's Nutri-Grain Apple Cinnamon Cereal", 'Nutri-Grain Soft Baked Strawberry Cereal Breakfast Bars', "Kellogg's Nutri-Grain Blueberry Cereal", 'Tiny Twists Pretzels', 'Traditional Snack Mix', 'Goldfish Cheddar Baked Snack Crackers', 'Original Orange Juice', 'Sugarfree Energy Drink', 'Energy Drink'], ['Bag of Organic Bananas', 'Just Crisp, Parm

In [56]:
# Limit transactions to the first 10000 rows
transactions = transactions[:5000]

In [57]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

df.head()

,& Go! Hazelnut Spread + Pretzel Sticks,0 Calorie Strawberry Dragonfruit Water Beverage,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Fat Superfruits Greek Yogurt,0% Greek Strained Yogurt,0% Greek Yogurt Black Cherry on the Bottom,"0% Greek, Blueberry on the Bottom Yogurt",0% Milkfat Greek Plain Yogurt,...,gel hand wash sea minerals,in Gravy with Carrots Peas & Corn Mashed Potatoes & Meatloaf Nuggets,of Hanover 100 Calorie Pretzels Mini,of Norwich Original English Mustard Powder Double Superfine,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,with Crispy Almonds Cereal,with Olive Oil Mayonnaise,with Olive Oil Mayonnaise Dressing,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [37]:
# from mlxtend.frequent_patterns import apriori, association_rules

# # Get frequent itemsets
# frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)

# # Generate rules
# rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)

# # Sort by lift
# rules = rules.sort_values(by="lift", ascending=False)

# print(rules.head(10))


In [58]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

frequent_itemsets = fpgrowth(df, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
rules = rules.sort_values(by="confidence", ascending=False)

In [59]:
print(rules.head(100))

                 antecedents               consequents  antecedent support  \
22        (Honeycrisp Apple)                  (Banana)              0.0286   
27      (Organic Fuji Apple)                  (Banana)              0.0294   
24          (Cucumber Kirby)                  (Banana)              0.0276   
10     (Organic Raspberries)  (Bag of Organic Bananas)              0.0448   
6     (Organic Hass Avocado)  (Bag of Organic Bananas)              0.0666   
29            (Strawberries)                  (Banana)              0.0438   
19         (Organic Avocado)                  (Banana)              0.0550   
20      (Organic Whole Milk)                  (Banana)              0.0392   
14    (Organic Strawberries)                  (Banana)              0.0838   
13     (Organic Raspberries)    (Organic Strawberries)              0.0448   
0     (Organic Baby Spinach)                  (Banana)              0.0700   
16    (Organic Strawberries)  (Bag of Organic Bananas)          

In [60]:
def recommend_products(cart_items, rules, top_n=5):
    """
    Recommend products based on unseen cart items using association rules.
    
    cart_items : list of str
        Products currently in the user's cart
    rules : pd.DataFrame
        Association rules dataframe from mlxtend
    top_n : int
        Number of recommendations to return
    """
    recommendations = []

    for item in cart_items:
        # Find rules where this item appears in the antecedents
        matched_rules = rules[rules['antecedents'].apply(lambda x: item in x)]
        
        for _, row in matched_rules.iterrows():
            for consequent in row['consequents']:
                if consequent not in cart_items:  # avoid suggesting same item
                    recommendations.append((consequent, row['confidence'], row['lift']))
    
    # Sort by confidence, then lift
    recommendations = sorted(recommendations, key=lambda x: (x[1], x[2]), reverse=True)

    # Keep only unique top_n items
    seen, final_recs = set(), []
    for rec in recommendations:
        if rec[0] not in seen:
            final_recs.append(rec)
            seen.add(rec[0])
        if len(final_recs) >= top_n:
            break

    return final_recs


In [ ]:
# Example unseen cart
cart = ["Peanut Butter", "Bag of Organic Bananas"]

# Get recommendations
recs = recommend_products(cart, rules, top_n=5)

print("Cart:", cart)
print("\nRecommended Products:")
for item, conf, lift in recs:
    print(f"- {item} (confidence={conf:.2f}, lift={lift:.2f})")


Cart: ['Bag of Organic Bananas', 'Peanut Butter']

Recommended Products:
- Organic Hass Avocado (confidence=0.17, lift=2.61)
- Organic Strawberries (confidence=0.14, lift=1.69)
- Organic Raspberries (confidence=0.12, lift=2.73)
- Organic Baby Spinach (confidence=0.11, lift=1.61)


Just to determine stronger rules

In [39]:
# Define thresholds
min_support = 0.005     # 0.5% of transactions
min_confidence = 0.2    # at least 20% chance
min_lift = 1.2          # rule is better than random chance

# Apply filtering
strong_rules = rules[
    (rules['support'] >= min_support) &
    (rules['confidence'] >= min_confidence) &
    (rules['lift'] >= min_lift)
].sort_values(by=['lift','confidence'], ascending=False)

print(strong_rules[['antecedents','consequents','support','confidence','lift']].head(20))


               antecedents               consequents  support  confidence  \
15   (Organic Raspberries)    (Organic Strawberries)   0.0100    0.233100   
22      (Honeycrisp Apple)                  (Banana)   0.0102    0.387833   
12   (Organic Raspberries)  (Bag of Organic Bananas)   0.0131    0.305361   
25    (Organic Fuji Apple)                  (Banana)   0.0102    0.364286   
6   (Organic Hass Avocado)  (Bag of Organic Bananas)   0.0189    0.294852   
27          (Strawberries)                  (Banana)   0.0133    0.287257   
21       (Organic Avocado)                  (Banana)   0.0159    0.278947   
4   (Organic Baby Spinach)  (Bag of Organic Bananas)   0.0157    0.216552   
18  (Organic Strawberries)  (Bag of Organic Bananas)   0.0170    0.215463   
16  (Organic Strawberries)                  (Banana)   0.0187    0.237009   
1   (Organic Baby Spinach)                  (Banana)   0.0149    0.205517   

        lift  
15  2.954376  
22  2.601158  
12  2.525735  
25  2.443231  


In [40]:
def frozenset_to_str(fset):
    return ', '.join(list(fset))

lookup_df = strong_rules.copy()
lookup_df['antecedents'] = lookup_df['antecedents'].apply(frozenset_to_str)
lookup_df['consequents'] = lookup_df['consequents'].apply(frozenset_to_str)

# Keep only useful columns
lookup_df = lookup_df[['antecedents','consequents','support','confidence','lift']]


In [42]:
lookup_df.head()

,antecedents,consequents,support,confidence,lift
15,Organic Raspberries,Organic Strawberries,0.0100,0.233100,2.954376
22,Honeycrisp Apple,Banana,0.0102,0.387833,2.601158
12,Organic Raspberries,Bag of Organic Bananas,0.0131,0.305361,2.525735
25,Organic Fuji Apple,Banana,0.0102,0.364286,2.443231
6,Organic Hass Avocado,Bag of Organic Bananas,0.0189,0.294852,2.438807


In [ ]:
# lookup_df.to_csv("product_recommendations.csv", index=False)
# print("CSV saved: product_recommendations.csv")

CSV saved: product_recommendations.csv
